In [ ]:
from __future__ import annotations
import math
from dataclasses import dataclass
from typing import Iterable, Iterator, Protocol, runtime_checkable

print("ready")

## 1) Mental model: identity, equality, mutability

Two orthogonal questions:

- **Identity**: are these two references to the same object? (`is`)
- **Equality**: do these two objects represent the same value? (`==`)

Mutability determines whether ‘value’ can change over time.


In [ ]:
a = [1, 2, 3]
b = a
c = [1, 2, 3]

print(a is b, a == b)
print(a is c, a == c)

a.append(4)
print("b sees mutation:", b)
print("c is separate:", c)

## 2) Designing a class: invariants and representation

An **invariant** is a condition that must always be true for a valid instance.

Example invariant: a 2D vector’s components are finite real numbers.

We’ll implement `Vector2` with:

- explicit validation (fail fast)
- nice string representation (`__repr__`)
- value semantics (`__eq__`)
- hashing when immutable (so it can be used as dict keys)

We’ll use `@dataclass(frozen=True)` to express immutability.


In [ ]:
@dataclass(frozen=True, slots=True)
class Vector2:
    x: float
    y: float

    def __post_init__(self) -> None:
        # object is frozen; must use object.__setattr__ if we needed to normalize, etc.
        if not (math.isfinite(self.x) and math.isfinite(self.y)):
            raise ValueError("Vector2 components must be finite")

    def norm(self) -> float:
        return math.hypot(self.x, self.y)

    def normalized(self) -> "Vector2":
        n = self.norm()
        if n == 0:
            raise ValueError("cannot normalize zero vector")
        return Vector2(self.x / n, self.y / n)

    def __add__(self, other: "Vector2") -> "Vector2":
        if not isinstance(other, Vector2):
            return NotImplemented
        return Vector2(self.x + other.x, self.y + other.y)

    def __mul__(self, scalar: float) -> "Vector2":
        if not isinstance(scalar, (int, float)):
            return NotImplemented
        return Vector2(self.x * float(scalar), self.y * float(scalar))

    __rmul__ = __mul__


v = Vector2(3.0, 4.0)
print(v, "norm=", v.norm())
print("hashable:", {v: "ok"}[v])

### Design note: `NotImplemented` is a feature

Returning `NotImplemented` from binary ops tells Python to try the reflected operation on the other operand.

This is cleaner than raising `TypeError` yourself in many cases.

Try `Vector2(...) + 1` and observe the error message.


In [ ]:
try:
    print(Vector2(1, 2) + 1)
except TypeError as e:
    print("TypeError:", e)

## 3) Attribute access & the data model (practical view)

Key points:

- `obj.attr` triggers `obj.__getattribute__('attr')`.
- If not found, it may fall back to `__getattr__`.
- Functions stored on a class are **descriptors**: they bind `self` to produce methods.

We’ll quickly inspect the method binding behavior.


In [ ]:
class Demo:
    def f(self, x):
        return ("self", self, "x", x)


d = Demo()
print("From instance:", d.f(10))
print("From class (unbound):", Demo.f(d, 10))
print("d.f is a bound method object:", d.f)

## 4) Protocols and structural typing (lightweight interface)

In Python, you often want to accept ‘anything that behaves like X’.

`typing.Protocol` lets you specify that contract without inheritance.


In [ ]:
@runtime_checkable
class SupportsNorm(Protocol):
    def norm(self) -> float: ...


def unit_length(obj: SupportsNorm) -> bool:
    return abs(obj.norm() - 1.0) < 1e-9


print(unit_length(Vector2(1, 0)))
print(isinstance(Vector2(1, 0), SupportsNorm))

## 5) Iterators and generators (your own collections)

If a class implements `__iter__`, it becomes iterable.

Below: a tiny range-like type with a custom iterator.


In [ ]:
class CountTo:
    def __init__(self, n: int):
        if n < 0:
            raise ValueError("n must be >= 0")
        self._n = n

    def __iter__(self) -> Iterator[int]:
        i = 0
        while i <= self._n:
            yield i
            i += 1


print(list(CountTo(5)))

## 6) Context managers (resource safety)

Context managers are about **guaranteeing cleanup**.

You can implement them with `__enter__`/`__exit__`, or use `contextlib` helpers.

Here’s a minimal timer context manager.


In [ ]:
import time


class Timer:
    def __enter__(self):
        self._t0 = time.perf_counter()
        return self

    def __exit__(self, exc_type, exc, tb):
        self.elapsed = time.perf_counter() - self._t0
        # Returning False means exceptions propagate.
        return False


with Timer() as t:
    s = sum(i * i for i in range(200_000))
print("elapsed:", round(t.elapsed, 4), "seconds")

# Exercises (do these)

## Exercise A — `Vector` with richer behavior

Implement a `VectorN` class with:

- immutable storage (tuple)
- validation (finite floats)
- `__len__`, `__iter__`, `__getitem__`
- `dot(other)` and `norm()`
- rich comparison _only if_ you can justify an ordering (hint: ordering vectors is not canonical)

## Exercise B — `LRUCache` via `__getitem__` / `__setitem__`

Implement a small LRU cache (size `k`) with O(1) operations using `collections.OrderedDict`.

## Exercise C — Context manager

Write a context manager that records timings for multiple named blocks, e.g.:

```python
with profiler.section('load'): ...
with profiler.section('compute'): ...
```

and later prints a summary.


In [ ]:
# Starter for Exercise A
from dataclasses import dataclass


@dataclass(frozen=True, slots=True)
class VectorN:
    data: tuple[float, ...]

    def __post_init__(self):
        if len(self.data) == 0:
            raise ValueError("VectorN cannot be empty")
        if not all(
            isinstance(x, (int, float)) and math.isfinite(float(x)) for x in self.data
        ):
            raise ValueError("all components must be finite real numbers")
        object.__setattr__(self, "data", tuple(float(x) for x in self.data))

    def __len__(self):
        return len(self.data)

    def __iter__(self):
        return iter(self.data)

    def __getitem__(self, i: int) -> float:
        return self.data[i]

    def dot(self, other: "VectorN") -> float:
        if len(self) != len(other):
            raise ValueError("dimension mismatch")
        return sum(a * b for a, b in zip(self, other))

    def norm(self) -> float:
        return math.sqrt(self.dot(self))


v = VectorN((3, 4))
print(v.norm())